In [196]:
from langchain_community.llms import Ollama, ollama

"""
Tool supported:
+llama3:instruct
+

NOT: 
+gemma:instruct: need to pare result to fix

"""

model_name = "llama3:instruct"
model_name = "gemma:instruct"
model_name = "wizardlm2"
model_name = "phi3:instruct"
model_name = "qwen:7b-chat"

param = {    
    "num_keep": 24,
    "stop": [
        "<|start_header_id|>",
        "<|end_header_id|>",
        "<|eot_id|>"
    ],
    "num_ctx": 512,
    "temperature": 0.0, 
}
param = {
    "penalize_newline": False,
    "repeat_penalty": 1,
    "stop": [
        "<start_of_turn>",
        "<end_of_turn>"
    ]
}
param = {
    "num_keep": 4,
    "stop": [
        "<|user|>",
        "<|assistant|>",
        "<|system|>",
        "<|end|>",
        "<|endoftext|>"
    ]
}
param = {
    "stop": [
        "<|im_start|>",
        "<|im_end|>"
    ]
}

model = Ollama(model = model_name, temperature=0.0, stop = param["stop"])#repeat_penalty = param["repeat_penalty"])#stop=stop)

# print(model(msg))

In [197]:
model("你哈")

'你好！有什么问题或者需要帮助的吗？\n'

In [200]:
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
# from langchain_openai import OpenAI
from langchain.tools import BaseTool, StructuredTool, tool
from langchain_core.runnables import RunnableLambda

# model = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.0)


# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")
    punchline_a: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field


def gemma(query: str) -> str:
    """Look up things online."""
    return query.replace("```json\n", "'''").replace("```","'''").replace(",\n}", "\n}").replace("'","")

# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# And a query intended to prompt a language model to populate the data structure.
# prompt_and_model = prompt | model | RunnableLambda(gemma) #| parser
prompt_and_model = prompt | model #| parser
# output = prompt_and_model.invoke({"query": "Tell me a joke."})
output = prompt_and_model.invoke({"query": "讲个笑话."})
output
# parser.invoke(output)

'根据您提供的输出schema，我可以生成一个笑话的JSON实例。以下是例子：\n\n```json\n{\n  "setup": "为什么有些人不喜欢数学？",\n  "punchline": "因为他们觉得答案总是‘boring’！",\n  "punchline_a": ""\n}\n```\n\n这个笑话以数学为背景，通过“答案总是‘boring’”的幽默回答来结束。\n'

In [179]:
output
import json
json.loads(output.replace(",\n}", "\n}").replace("'",""))
# parser.invoke("".join(y))

JSONDecodeError: Expecting property name enclosed in double quotes: line 4 column 1 (char 120)